In [1]:
import pandas as pd
import re

In [2]:
# Change this path if needed
df1 = pd.read_csv("C:/Users/sayan/Downloads/imdb_query_execution_times_complex_10k.csv")
df2 = pd.read_csv("C:/Users/sayan/Downloads/imdb_query_execution_times_complex_mixed_10k.csv")
df3 = pd.read_csv("C:/Users/sayan/Downloads/imdb_query_execution_times_complex_added_10k.csv")
df = pd.concat([df1, df2, df3], ignore_index=True)
# Show first few rows
print(df.head())

                                               query  execution_time
0  SELECT tb.tconst, tb.primaryTitle, tr.averageR...        0.003061
1  SELECT nb.primaryName, tb.primaryTitle FROM na...        0.000158
2  SELECT nb.primaryName, AVG(tr.averageRating) A...        0.000105
3  SELECT nb.primaryName AS directorName, COUNT(*...        0.000158
4  SELECT te.tconst, te.parentTconst, tb.primaryT...        0.000150


In [3]:
sql_keywords = [
    # Core DML / DDL
    "SELECT", "INSERT", "UPDATE", "DELETE", "CREATE", "ALTER", "DROP", "TRUNCATE",
    "FROM", "WHERE", "GROUP BY", "ORDER BY", "HAVING", "LIMIT", "OFFSET",
    "DISTINCT", "AS", "INTO", "VALUES", "SET",

    # Joins
    "JOIN", "INNER JOIN", "LEFT JOIN", "RIGHT JOIN", "FULL JOIN", "CROSS JOIN",
    "UNION", "UNION ALL", "INTERSECT", "EXCEPT",

    # Aggregates
    "COUNT", "SUM", "AVG", "MIN", "MAX",

    # Conditions
    "AND", "OR", "NOT", "IN", "BETWEEN", "LIKE", "IS NULL", "IS NOT NULL",
    "EXISTS", "CASE", "WHEN", "THEN", "ELSE", "END",

    # Window / set
    "ALL", "ANY", "SOME", "OVER", "PARTITION BY", "ROW_NUMBER", "RANK", "DENSE_RANK",

    # Schema / transaction
    "DATABASE", "TABLE", "INDEX", "VIEW", "TRIGGER", "SEQUENCE",
    "PRIMARY KEY", "FOREIGN KEY", "CHECK", "DEFAULT", "CONSTRAINT",
    "BEGIN", "COMMIT", "ROLLBACK",

    # Misc
    "EXPLAIN", "ANALYZE", "DESCRIBE", "SHOW", "USE"
]


In [4]:
def extract_sql_features(query):
    if pd.isna(query):
        return {}
    q = str(query).upper()
    features = {}

    # One-hot encode all keywords
    for kw in sql_keywords:
        kw_regex = r"\b" + re.escape(kw) + r"\b"
        features[kw.replace(" ", "_")] = int(bool(re.search(kw_regex, q)))

    # Number of attributes
    match = re.search(r"SELECT (.*?) FROM", q, re.DOTALL)
    if match:
        attrs = [a.strip() for a in match.group(1).split(',')]
        features["num_attributes"] = len([a for a in attrs if a and a != '*'])
    else:
        features["num_attributes"] = 0

    # Number of tables
    match = re.search(r"FROM (.*?)( WHERE| GROUP BY| ORDER BY|;|$)", q, re.DOTALL)
    if match:
        tables = [t.strip() for t in match.group(1).split(',')]
        features["num_tables"] = len([t for t in tables if t])
    else:
        features["num_tables"] = 0

    # Number of conditions
    features["num_conditions"] = len(re.findall(r"\bWHERE\b|\bAND\b|\bOR\b", q))

    # Number of joins
    features["num_joins"] = len(re.findall(r"\bJOIN\b", q))

    # Subquery flag
    features["has_subquery"] = int(len(re.findall(r"\(SELECT ", q)) > 0)

    # Total keyword count
    features["num_keywords"] = sum(
        features[k] for k in features if k in [kw.replace(" ", "_") for kw in sql_keywords]
    )

    return features


In [5]:
df["features"] = df["query"].apply(extract_sql_features)

# Expand the dict column into separate columns
features_df = pd.DataFrame(df["features"].tolist())

# Combine with the original dataset
final_df = pd.concat([df, features_df], axis=1)

In [6]:
len(final_df)

30000

In [71]:
x = final_df.drop(columns=["query", "execution_time", "features"], axis=1)
y = final_df["execution_time"]

In [72]:
x

,SELECT,INSERT,UPDATE,DELETE,CREATE,ALTER,DROP,TRUNCATE,FROM,WHERE,...,ANALYZE,DESCRIBE,SHOW,USE,num_attributes,num_tables,num_conditions,num_joins,has_subquery,num_keywords
0,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,3,1,1,1,0,6
1,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,2,1,2,2,0,6
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,2,1,0,2,0,7
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,2,1,0,2,0,7
4,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,4,1,1,2,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,3,1,2,1,0,13
29996,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,2,1,1,1,0,6
29997,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,2,1,3,2,0,8
29998,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,2,1,0,1,0,7


In [73]:
import numpy as np

In [74]:
x = np.array(x)
y = np.array(y)

In [75]:
len(x), len(x[0])

(30000, 82)

In [81]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()
y_scaled

array([ 2.2716212 , -0.0486227 , -0.0910724 , ..., -0.16128022,
       -0.15667051, -0.11586343])

In [124]:
y

array([3.06073400e-03, 1.57976000e-04, 1.04869000e-04, ...,
       1.70349999e-05, 2.28020003e-05, 7.38540002e-05])

In [125]:
scaler.inverse_transform(y_scaled.reshape(-1, 1)).flatten()

array([3.06073400e-03, 1.57976000e-04, 1.04869000e-04, ...,
       1.70349999e-05, 2.28020003e-05, 7.38540002e-05])

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [82]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y_scaled, test_size=0.2, random_state=42
)

In [83]:
X_train.shape, X_test.shape

((24000, 82), (6000, 82))

In [84]:
y_train[0]

-0.16745259531912512

In [85]:
def transformer_encoder(inputs, num_heads, key_dim, ff_dim, dropout=0.1):
    # Multi-head attention
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=dropout)(inputs, inputs)
    attn_output = layers.Dropout(dropout)(attn_output)
    out1 = layers.LayerNormalization(epsilon=1e-6)(inputs + attn_output)

    # Feed-forward network
    ffn = keras.Sequential([
        layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(1e-4)),
        layers.Dense(inputs.shape[-1])
    ])
    ffn_output = ffn(out1)
    ffn_output = layers.Dropout(dropout)(ffn_output)
    return layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

# =========================================
# 5️⃣ Build Tuned Transformer Model
# =========================================
input_shape = (X_train.shape[1], 1)
inputs = keras.Input(shape=input_shape)

# Project inputs to higher dimension
x = layers.Dense(128, activation="silu")(inputs)

# Stack 4 moderate transformer blocks
for _ in range(4):
    x = transformer_encoder(x, num_heads=4, key_dim=32, ff_dim=128, dropout=0.3)

# Global pooling
x = layers.GlobalAveragePooling1D()(x)

# Dense regression head with regularization
x = layers.Dense(128, activation="silu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(64, activation="silu")(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(32, activation="silu")(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs, outputs)

In [86]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="mse",
    metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.RootMeanSquaredError()]
)

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 82, 1)]      0           []                               
                                                                                                  
 dense_87 (Dense)               (None, 82, 128)      256         ['input_7[0][0]']                
                                                                                                  
 multi_head_attention_29 (Multi  (None, 82, 128)     66048       ['dense_87[0][0]',               
 HeadAttention)                                                   'dense_87[0][0]']               
                                                                                                  
 dropout_69 (Dropout)           (None, 82, 128)      0           ['multi_head_attention_29[0

In [87]:
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train.reshape(X_train.shape[0], X_train.shape[1], 1),
    y_train,
    validation_split=0.2,
    epochs=1000,
    batch_size=64,
    verbose=1
)


Epoch 1/1000
300/300 [==============================] - 9s 21ms/step - loss: 0.9634 - mean_absolute_error: 0.3822 - root_mean_squared_error: 0.9552 - val_loss: 1.7877 - val_mean_absolute_error: 0.9004 - val_root_mean_squared_error: 1.3179
Epoch 2/1000
300/300 [==============================] - 8s 28ms/step - loss: 0.5480 - mean_absolute_error: 0.3005 - root_mean_squared_error: 0.7051 - val_loss: 0.9742 - val_mean_absolute_error: 0.4668 - val_root_mean_squared_error: 0.9609
Epoch 3/1000
300/300 [==============================] - 12s 41ms/step - loss: 0.3711 - mean_absolute_error: 0.2340 - root_mean_squared_error: 0.5660 - val_loss: 0.4895 - val_mean_absolute_error: 0.3752 - val_root_mean_squared_error: 0.6625
Epoch 4/1000
300/300 [==============================] - 12s 41ms/step - loss: 0.3328 - mean_absolute_error: 0.2108 - root_mean_squared_error: 0.5312 - val_loss: 0.2700 - val_mean_absolute_error: 0.2063 - val_root_mean_squared_error: 0.4684
Epoch 5/1000
300/300 [====================

KeyboardInterrupt: 

In [88]:
test_results = model.evaluate(
    X_test.reshape(X_test.shape[0], X_test.shape[1], 1), y_test, verbose=1
)
print("\n✅ Test results:", test_results)

188/188 [==============================] - 2s 10ms/step - loss: 0.2077 - mean_absolute_error: 0.0914 - root_mean_squared_error: 0.4554

✅ Test results: [0.20770390331745148, 0.09142637997865677, 0.45544102787971497]


In [113]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [91]:
scaler.inverse_transform(np.array(test_results).reshape(-1, 1))

array([[0.00047866],
       [0.00033319],
       [0.00078859]])

In [135]:
pred = model.predict(
    X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
)
pred

188/188 [==============================] - 1s 8ms/step


array([[-0.14037518],
       [-0.1636066 ],
       [-0.16780655],
       ...,
       [-0.16195945],
       [-0.16360311],
       [-0.14090724]], dtype=float32)

In [140]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def evaluate_model(y_true_scaled, y_pred_scaled, scaler):
    """
    Compute MAE, RMSE, R2, and MAPE after inverse-transforming scaled predictions and targets.

    Parameters:
        y_true_scaled (array): Scaled true target values
        y_pred_scaled (array): Scaled predicted target values
        scaler (object): The scaler fitted on the training target (e.g. StandardScaler, MinMaxScaler)
    
    Returns:
        dict: MAE, RMSE, R2, and MAPE in original target units
    """
    # Inverse transform
    y_true = scaler.inverse_transform(y_true_scaled.reshape(-1, 1)).flatten()
    y_pred = scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    
    # Metrics
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # in percentage
    
    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MAPE (%)": mape}
metrics = evaluate_model(y_test, pred, scaler)
metrics

{'MAE': 0.00011437961911796012,
 'RMSE': 0.0005697828221060879,
 'R2': 0.799185425036971,
 'MAPE (%)': 67.9416715386394}

In [ ]:
from sklearn.linear_model import LinearRegression


In [138]:
yy = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
yytest = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

In [121]:
yy.shape

(24000,)

In [141]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, yy)

y_pred_lin = lin_reg.predict(X_test)

mae_lin = mean_absolute_error(yytest, y_pred_lin)
rmse_lin = mean_squared_error(yytest, y_pred_lin)
r2_lin = r2_score(yytest, y_pred_lin)
mape = np.mean(np.abs((yytest - y_pred_lin) / yytest)) * 100  # in percentage

print("\n📊 Linear Regression Results:")
print(f"MAE  : {mae_lin:.4f}")
print(f"RMSE : {np.sqrt(rmse_lin):.4f}")
print(f"MAPE : {mape:.4f}%")

print(f"R²   : {r2_lin:.4f}")



📊 Linear Regression Results:
MAE  : 0.0001
RMSE : 0.0003
MAPE : 52.3701%
R²   : 0.9298


In [92]:
mae_lin

5.7560062904610826e-05